In [ ]:
!pip install pandas sqlalchemy psycopg2

In [68]:
import pandas as pd
from sqlalchemy import create_engine, text

DB_USER = "postgres"
DB_PASSWORD = "root"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "green-taxi-data"

CSV_FILE = "data/green_tripdata_2019-10.csv"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS green_trips CASCADE;"))
    conn.execute(text("CREATE TABLE green_trips (LIKE green_taxi_data INCLUDING ALL);"))

chunksize = 100000  
dtype = {
    'VendorID': 'Int64',
    'passenger_count': 'Int64',
    'trip_distance': 'float64',
    'RatecodeID': 'Int64',
    'store_and_fwd_flag': 'str',
    'PULocationID': 'Int64',
    'DOLocationID': 'Int64',
    'payment_type': 'Int64',
    'fare_amount': 'float64',
    'extra': 'float64',
    'mta_tax': 'float64',
    'tip_amount': 'float64',
    'tolls_amount': 'float64',
    'improvement_surcharge': 'float64',
    'total_amount': 'float64',
    'congestion_surcharge': 'float64'
}

for chunk in pd.read_csv(
    CSV_FILE,
    chunksize=chunksize,
    dtype=dtype,  # <-- Ajouté
    parse_dates=['lpep_pickup_datetime', 'lpep_dropoff_datetime']
):
    chunk.to_sql("green_taxi_data", engine, if_exists="append", index=False)
    print(f"Ingested {len(chunk)} rows...")

CSV_FILE = "taxi_zone_lookup.csv"

df = pd.read_csv(CSV_FILE)

df.to_sql(
    name="taxi_zones", 
    con=engine,         
    if_exists="replace", 
    index=False      
)

print("Ingestion done !")

Ingested 100000 rows...
Ingested 100000 rows...
Ingested 100000 rows...
Ingested 100000 rows...
Ingested 76386 rows...
Ingestion done !


In [ ]:
query = """
    SELECT 
    COUNT(*) FILTER (WHERE trip_distance <= 1) AS up_to_1_mile,
    COUNT(*) FILTER (WHERE trip_distance > 1 AND trip_distance <= 3) AS between_1_3,
    COUNT(*) FILTER (WHERE trip_distance > 3 AND trip_distance <= 7) AS between_3_7,
    COUNT(*) FILTER (WHERE trip_distance > 7 AND trip_distance <= 10) AS between_7_10,
    COUNT(*) FILTER (WHERE trip_distance > 10) AS over_10
FROM green_taxi_data
WHERE 
    lpep_pickup_datetime >= '2019-10-01' AND 
    lpep_pickup_datetime < '2019-11-01';
"""

pd.read_sql(query, engine)

In [ ]:
query = """
    SELECT 
    DATE(lpep_pickup_datetime) AS pickup_day,
    MAX(trip_distance) AS longest_trip_distance
FROM green_trips
GROUP BY pickup_day
ORDER BY longest_trip_distance DESC
LIMIT 1;
"""

pd.read_sql(query, engine)

In [ ]:
query = """
    SELECT 
    tz."Zone" AS pickup_zone,
    SUM(gt.total_amount) AS total_amount
FROM green_trips gt
JOIN taxi_zones tz ON gt."PULocationID" = tz."LocationID"
WHERE DATE(gt.lpep_pickup_datetime) = '2019-10-18'
GROUP BY tz."Zone"
HAVING SUM(gt.total_amount) > 13000
ORDER BY total_amount DESC
LIMIT 3;
"""

pd.read_sql(query, engine)

In [ ]:
query = """
    SELECT 
    tz_drop."Zone" AS dropoff_zone,
    MAX(gt.tip_amount) AS largest_tip
FROM green_trips gt
JOIN taxi_zones tz_pick ON gt."PULocationID" = tz_pick."LocationID"
JOIN taxi_zones tz_drop ON gt."DOLocationID" = tz_drop."LocationID"
WHERE 
    tz_pick."Zone" = 'East Harlem North' AND
    DATE_TRUNC('month', gt.lpep_pickup_datetime) = '2019-10-01'
GROUP BY tz_drop."Zone"
ORDER BY largest_tip DESC
LIMIT 1;
"""

pd.read_sql(query, engine)